# Convert color images to grayscale

Transform RGB images to grayscale for analysis, preprocessing, or model inputs that require single-channel images.

**What's in this recipe:**
- Simple conversion with PIL
- Perceptually accurate grayscale (weighted RGB channels)
- Custom UDF for advanced conversion


## Problem

You need to convert color images to grayscale for analysis, preprocessing, or model inputs that require single-channel images.

Different conversion methods produce different results—you need to choose the right approach for your use case.


## Solution

**Without Pixeltable:** Write a loop to convert images with Pillow's `.convert()` (or OpenCV, NumPy), manage files yourself.

**With Pixeltable:** Use PIL's `.convert('L')` or custom UDFs in computed columns. All conversions automatic.

**Conversion methods:**

| Method | Speed | Accuracy | When to use |
|--------|-------|----------|-------------|
| **Simple (PIL `.convert('L')`)** | Fast | Good | Model preprocessing, general analysis |
| **Gamma-corrected (custom UDF)** | Slow | Best | Scientific imaging, professional photography |

The simple method uses PIL's built-in conversion. The gamma-corrected method requires a custom UDF (not built into PIL) that applies perceptual weighting in linear color space.

*For technical details on gamma correction and grayscale conversion, see [Wikipedia: Grayscale](https://en.wikipedia.org/wiki/Grayscale).*

### Setup


In [ ]:
%pip install -qU pixeltable numpy

In [ ]:
import pixeltable as pxt
import numpy as np
from PIL import Image

# Create a fresh directory (drop existing if present)
pxt.drop_dir('image_demo', force=True)
pxt.create_dir('image_demo')

### Load images


In [ ]:
t = pxt.create_table('image_demo.gray', {'image': pxt.Image})
t.insert([
    {'image': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000001.jpg'},
    {'image': 'https://raw.githubusercontent.com/pixeltable/pixeltable/main/docs/resources/images/000000000016.jpg'},
])


### Simple conversion with PIL


In [ ]:
# Built-in PIL conversion (fast and good for most use cases)
t.add_computed_column(grayscale=t.image.convert('L'))

t.select(t.image, t.grayscale).show()


### Perceptually accurate conversion

For scientific imaging or when perception accuracy matters, use a custom UDF with gamma correction.

#### Define the UDF


In [ ]:
@pxt.udf
def rgb_to_gray_accurate(img: Image.Image) -> Image.Image:
    """Convert RGB to grayscale with full gamma correction.
    
    Most accurate but slower. Gamma-decompresses, applies perceptual weights
    in linear space, then re-compresses for display.
    """
    rgb = np.array(img).astype(np.float32) / 255.0
    
    # Gamma decompress: make pixel values perceptually linear
    rgb_lin = ((rgb + 0.055) / 1.055) ** 2.4
    rgb_lin = np.where(rgb <= 0.04045, rgb / 12.92, rgb_lin)
    
    # Apply perceptual weights in linear space
    gray_lin = (
        0.2126 * rgb_lin[:, :, 0] +
        0.7152 * rgb_lin[:, :, 1] +
        0.0722 * rgb_lin[:, :, 2]
    )
    
    # Gamma compress: make values display-ready
    gray = 1.055 * gray_lin ** (1 / 2.4) - 0.055
    gray = np.where(gray_lin <= 0.0031308, 12.92 * gray_lin, gray)
    
    gray = (gray * 255).astype(np.uint8)
    return Image.fromarray(gray)

#### Test the conversion


In [ ]:
# Compare both methods on first image
t.select(
    t.image,
    t.grayscale,
    rgb_to_gray_accurate(t.image)
).head(1)


#### Apply to all images


In [ ]:
t.add_computed_column(accurate=rgb_to_gray_accurate(t.image))

# View all results
t.select(t.image, t.grayscale, t.accurate).show()


## Explanation

**Two approaches:**

1. **Simple (`.convert('L')`):** PIL's built-in. Fast, good for most use cases (model preprocessing, general analysis).

2. **Gamma-corrected (custom UDF):** Not built into PIL. Requires a custom UDF that:
   - Gamma-decompresses to linear space
   - Applies perceptual weights: 0.2126 × R + 0.7152 × G + 0.0722 × B
   - Gamma-compresses back for display
   - Slower but most perceptually accurate
   - Use for scientific imaging, professional photography

**Why gamma matters:** Displays aren't linear—doubling a pixel value doesn't double perceived brightness. Gamma correction accounts for this. For best results, convert to linear space before weighting, then convert back.

*The gamma-corrected method is based on [Brandon Rohrer's explanation](https://brandonrohrer.com/convert_rgb_to_grayscale.html) of perceptually accurate RGB to grayscale conversion.*


## See also

- [Transform images with PIL operations](./image-transformations.ipynb)
- [Test transformations with fast feedback loops](../iteration/fast-feedback-loops.ipynb)
